In [14]:
import json

import pickle
import numpy as np
import jionlp as jio
import numpy as np
import matplotlib.pyplot as plt

In [15]:
def load_data_json(json_path):
    f = open(json_path, 'r', encoding='utf-8')
    json_data = json.load(f)
    f.close()
    return json_data

In [16]:
import pandas as pd
import glob
files = glob.glob("./data/Keywords/*.xlsx")
# files = glob.glob("./data/Keywords/02pest.list.xlsx")
# files += glob.glob("./data/Keywords/02crop.list.xlsx")
same_words_dict = {}

for file in files:
    df = pd.read_excel(file, header=None)
    df_list = df.values.tolist()
    for words in df_list:
        change_to = words[0]
        same_words_dict[change_to] = change_to
        for word in words:
            word = str(word)
            if word == 'nan':
                break
            if word != change_to:
                same_words_dict[word] = change_to
                

In [17]:
json_path = f'./data/train_complete.json'
json_data = load_data_json(json_path)

results_path = f'./output_results/train_result_colbert_text_keywords_hard_neg5_rand_neg30_epoch2'
with open(results_path, 'rb') as handle:
    results = pickle.load(handle)
print(len(results))    

559


In [18]:
binary_result_path  = f'./output_results/train_binary_result_cross_encoder_title_keyword_epoch5'
with open(binary_result_path, 'rb') as handle:
    binary_results = pickle.load(handle)

In [19]:
print(len(binary_results))

560


In [20]:
print(len(json_data))
print(results['1'][:3])

559
[('3', -42.186592), ('5', -41.89062), ('6', -40.656452)]


In [21]:
def softmax(x):
    """Compute the softmax in a numerically stable way."""
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [22]:
from sklearn.metrics import f1_score, recall_score, precision_score

from sklearn.preprocessing import MultiLabelBinarizer
multi = MultiLabelBinarizer()

all_dids = []
did2data = {}
for d in json_data:
    did = d['did']
    did2data[did] = d
    all_dids.append(did)


In [10]:
from gensim.summarization import bm25
corpus_ids = [d['did'] for d in json_data]
corpus = []
titles = []
title_and_corpus = []

from zhon.hanzi import punctuation
import string
for i, d in enumerate(json_data):
    text_words = d['replaced_text_sentence'].split()
    title_words = d['replaced_title_sentence'].split()

    filter_title_words = [w for w in title_words if w not in punctuation and w not in string.punctuation]

    filter_text_words = [w for w in text_words if w not in punctuation and w not in string.punctuation]

    corpus.append(filter_text_words)
    titles.append(filter_title_words)
    title_and_corpus.append(filter_title_words + filter_text_words)

bm25Model = bm25.BM25(title_and_corpus)

In [11]:
for i, query in enumerate((title_and_corpus)):
    q_did = corpus_ids[i]
    result = results[q_did]
#     bm25_scores = []
    scores = bm25Model.get_scores(query)
    for idx, score in enumerate(scores):
        ref_did = corpus_ids[idx]
        if q_did == ref_did:
            continue
        for k, r in enumerate(result):
            if r[0] == ref_did:
                r = r + (score,)
                result[k] = r
                break
    results[q_did] = result


In [12]:
# # keyword_sets = []
# for i, d in enumerate(json_data):
#     did = d['did']
#     text_words = d['text_sentence'].split()
#     title_words = d['title_sentence'].split()
#     all_words = text_words + title_words
#     keyword_set = set()
#     for k, v in same_words_dict.items():
#         if k in all_words:
#             keyword_set.add(same_words_dict[k])
# #     text = ' '.join(all_words)
# #     key_phrases = jio.keyphrase.extract_keyphrase(text)
# #     for key_phrase in key_phrases:
# #         if key_phrase in same_words_dict:
# #             keyword_set.add(same_words_dict[key_phrase])
# #         else:
# #             keyword_set.add(key_phrase)
# #     keyword_sets.append(keyword_set)
#     json_data[i]['keyword_set'] = keyword_set
# #     print(did)
# #     print(keyword_set)
# #     input()
    
            

In [13]:
# keyword_sets = []
# for i, d in enumerate(json_data):
#     did = d['did']
#     text_words = d['text']
#     title_words = d['title']
#     all_texts = text_words + title_words
#     keyword_set = set()
#     for k, v in same_words_dict.items():
#         if k in all_texts:
# #             keyword_set.add(k)
#             keyword_set.add(v)
#     keyword_sets.append(keyword_set)
#     json_data[i]['keyword_set'] = keyword_set
    

In [14]:
def jaccard_score(set1, set2):
    u = set1.union(set2) 
    i = set1.intersection(set2)
    if len(u) == 0:
        return 0
    return len(i) / len(u)


def dsc_score(set1, set2):
    i = set1.intersection(set2)
    if len(set1) + len(set2) == 0:
        return 0
    return (2 * len(i)) / (len(set1) + len(set2))

def overlap_coefficient(set1, set2):
    i = set1.intersection(set2)
    min_ = min(len(set1), len(set2))
    if min_ == 0:
        return 0
    return len(i) / min_


In [15]:
from collections import defaultdict
keyword_set_sim = defaultdict(dict)
for i in range(len(json_data)):
    for k in range(len(json_data)):
        
        did_i = json_data[i]['did']
        did_k = json_data[k]['did']
        
        set_i = set(json_data[i]['keyword_set'])
        set_k = set(json_data[k]['keyword_set'])
        
        sim_socre = dsc_score(set_i, set_k)
        keyword_set_sim[did_i][did_k] = sim_socre

#         inter = set_i.intersection(set_k)
#         keyword_set_sim[did_i][did_k] = len(inter)

In [16]:
for i in range(len(json_data)):
    did_i = json_data[i]['did']
    result = results[did_i]
    
    
    set_sim_score = keyword_set_sim[did_i]
    for ref_did, score in set_sim_score.items():
        if did_i == ref_did:
            continue
        for k, r in enumerate(result):
            if r[0] == ref_did:
                r = r + (score,)
                result[k] = r
                break
    results[did_i] = result

In [17]:
def show_pos_score(pos_dids, scores, all_ref_ids):
    for pos_did in pos_dids:
        pos_did_score = scores[np.where(all_ref_ids==pos_did)[0]]
        print(f'pos_did : {pos_did} softmax score: {pos_did_score}')

In [18]:
def get_static_info(li):
    mean = np.mean(li)
    median = np.median(li)
    std = np.std(li)
    max_ = np.max(li)
    min_ = np.min(li)
    print(f' mean {mean} median {median}, std {std}, max {max_}, min {min_}')


In [19]:
topk = 300
all_pos_scores = []
all_scores = []
c = 0
avg_pos_hit_rate = 0

y = []
y_bm25 = []
y_pred = []
y_sets = []
y_second_pred = []
y_binary = []
y_mixed = []
y_score_pred = []
bm25_avg_length = 0
train_binary_pairs = []
train_pos_count = 0
train_neg_count = 0

second_num_of_std = 1.4
avg_pos_rate = 0.0
count = 0

num_of_std = 1
binary_th = 0.8
set_th = 0.5

for k, values in results.items():
    did = str(k)
    
    binary_value = binary_results[k]
    
    data = did2data[did]
    pos_dids = data['pos_dids']
    count += len(pos_dids)
    if len(pos_dids) == 0:
        continue
    
    all_ref_ids = np.array([v[0] for v in values])


    print(f'did = {k}')
    print('label pos dids')
    print(pos_dids)
#     bm25_pos_dids = data['bm25_pos_dids'][:topk]
    
    bm25_scores = [v[3] for v in values]
    bm25_scores = np.array(bm25_scores)
    
#     bm25_scores_normalize = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores))
    
    bm25_mean = np.mean(bm25_scores)
    bm25_median = np.median(bm25_scores)
    bm25_std = np.std(bm25_scores)
    bm25_scores_normalize = (bm25_scores - bm25_mean) / bm25_std
    bm25_max_score = np.max(bm25_scores)
    bm25_th = bm25_median + num_of_std * bm25_std
    
#     bm25_pos_idx = bm25_scores >= bm25_th
    bm25_pos_idx = bm25_scores_normalize >= 3.2
    bm25_preds = (all_ref_ids[bm25_pos_idx])
    bm25_pos_scores = bm25_scores[bm25_pos_idx]
    
    
#     binary_scores = [v[1] for v in binary_value]
#     binary_scores = np.array(binary_scores)
#     binary_pos_idx = binary_scores >= binary_th
    binary_preds = []
#     binary_preds = all_ref_ids[binary_pos_idx]
#     binary_pos_scores = binary_scores[binary_pos_idx]
    


    set_scores =  [v[3] for v in values]
    set_scores = np.array(set_scores)
#     set_scores_normalize = (set_scores - np.mean(set_scores)) / np.std(set_scores)
    set_pos_idx = set_scores >= 1
    set_preds = (all_ref_ids[set_pos_idx])
    set_pred_scores = (set_scores[set_pos_idx])
    
    

    
    scores =  [v[1] for v in values]
    scores = np.array(scores)
#     scores_normalize = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))
    mean = np.mean(scores)
    median = np.median(scores)
    std = np.std(scores)
    scores_normalize = (scores - mean) / std
    max_score = np.max(scores)
    th = median + num_of_std * std
    pos_idx = scores_normalize >= 5
    preds = all_ref_ids[pos_idx]
    pos_scores = scores[pos_idx]
    
    
    first_scores = bm25_pos_scores
    first_preds = bm25_preds
    
    secnod_pred_mean = np.mean(first_scores)
    second_median = np.median(first_scores)
    second_std = np.std(first_scores)
    second_th = second_median + second_num_of_std * second_std
    second_pos_idx = first_scores >=  second_th
    second_preds = first_preds[second_pos_idx]
    second_pos_scores = first_scores[second_pos_idx]
#     print(second_preds)
#     print(second_pos_scores)
    
#     alpha = 0.5
#     mix_scores = (alpha * set_scores) + ( (1 - alpha) * binary_scores)
#     mix_mean = np.mean(mix_scores)
#     mix_median = np.median(mix_scores)
#     mix_std = np.std(mix_scores)
#     mix_th = mix_mean + 1 * mix_std
#     mix_pos_idx = mix_scores >= 0.7
#     mix_preds = all_ref_ids[mix_pos_idx]
    mix_preds = []
#     print(mix_preds)

    
#     set_and_binary_scores = set_scores * binary

    
#     mix_scores = alpha * set_scores + (1-alpha) * binary
    
    
#     print(binary_scores.shape)

#     print(f' mean {mean}, std {std}, max {max_score}')
    
    be_check_scores = set_scores
    for pos_did in pos_dids :
        idx = np.where(all_ref_ids == pos_did)
        p_score = be_check_scores[idx]
        print(f'did : {all_ref_ids[idx]}, score : {p_score}')
        all_pos_scores.append(p_score[0])
    all_scores += list(be_check_scores)
    
#     scores = softmax(scores)
#     topk_idx = scores.argsort()[::-1][:topk]
    


    
#     print('score based predict topk did and score')
    
#     print(preds)
#     if len(pos_dids) > 0:
#         for pos_did in pos_dids:
#             train_binary_pairs.append((did, pos_did, 1))
#             train_pos_count += 1
#         for pred in preds:
#             if pred in pos_dids:
#                 continue
#             train_binary_pairs.append((did, pred, 0))
#             train_neg_count += 1
#         print(len(pos_dids), len(pred), len(pos_dids)/(len(pred) + len(pos_dids)))

    y.append(pos_dids)
    y_bm25.append(bm25_preds)
    y_pred.append(preds)
    y_second_pred.append(second_preds)
    y_sets.append(set_preds)
    y_binary.append(binary_preds)
    y_mixed.append(mix_preds)
    y_score_pred.append(preds)



did = 3
label pos dids
['649', '415']
did : ['649'], score : [0.35714286]
did : ['415'], score : [0.42857143]
did = 9
label pos dids
['5']
did : ['5'], score : [1.]
did = 25
label pos dids
['41', '32']
did : ['41'], score : [1.]
did : ['32'], score : [0.90909091]
did = 26
label pos dids
['37']
did : ['37'], score : [0.82352941]
did = 27
label pos dids
['46']
did : ['46'], score : [1.]
did = 29
label pos dids
['72']
did : ['72'], score : [1.]
did = 32
label pos dids
['41', '25']
did : ['41'], score : [0.90909091]
did : ['25'], score : [0.90909091]
did = 34
label pos dids
['40']
did : ['40'], score : [1.]
did = 35
label pos dids
['191', '75', '66']
did : ['191'], score : [0.30769231]
did : ['75'], score : [0.57142857]
did : ['66'], score : [1.]
did = 40
label pos dids
['34']
did : ['34'], score : [1.]
did = 41
label pos dids
['32', '25']
did : ['32'], score : [0.90909091]
did : ['25'], score : [1.]
did = 44
label pos dids
['34', '1049', '40', '806', '1054']
did : ['34'], score : [0.61538

did : ['222'], score : [0.77777778]
did = 333
label pos dids
['785', '713', '789']
did : ['785'], score : [0.53333333]
did : ['713'], score : [0.66666667]
did : ['789'], score : [0.64]
did = 342
label pos dids
['442', '850', '946', '93']
did : ['442'], score : [0.5]
did : ['850'], score : [0.32432432]
did : ['946'], score : [0.75]
did : ['93'], score : [0.625]
did = 351
label pos dids
['673', '747']
did : ['673'], score : [0.8]
did : ['747'], score : [0.82758621]
did = 361
label pos dids
['771', '1167']
did : ['771'], score : [0.82352941]
did : ['1167'], score : [0.93333333]
did = 362
label pos dids
['170']
did : ['170'], score : [0.66666667]
did = 365
label pos dids
['774', '380', '1126', '676', '326', '656', '382']
did : ['774'], score : [0.66666667]
did : ['380'], score : [0.94736842]
did : ['1126'], score : [0.66666667]
did : ['676'], score : [0.66666667]
did : ['326'], score : [0.66666667]
did : ['656'], score : [0.66666667]
did : ['382'], score : [0.88888889]
did = 371
label pos 

/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


did : ['654'], score : [1.]
did : ['620'], score : [1.]
did = 676
label pos dids
['774', '326', '656', '1126']
did : ['774'], score : [1.]
did : ['326'], score : [1.]
did : ['656'], score : [1.]
did : ['1126'], score : [1.]
did = 678
label pos dids
['732', '667', '659']
did : ['732'], score : [0.8]
did : ['667'], score : [0.92307692]
did : ['659'], score : [1.]
did = 682
label pos dids
['644']
did : ['644'], score : [1.]
did = 684
label pos dids
['709', '661', '636']
did : ['709'], score : [1.]
did : ['661'], score : [1.]
did : ['636'], score : [1.]
did = 689
label pos dids
['649', '415']
did : ['649'], score : [0.88888889]
did : ['415'], score : [0.83333333]
did = 690
label pos dids
['1256', '700']
did : ['1256'], score : [0.85714286]
did : ['700'], score : [0.85714286]
did = 691
label pos dids
['1119', '816', '478', '498', '790', '552', '646', '694', '984', '470', '616', '129', '759', '375', '46', '95', '586', '609', '163', '476', '344', '986', '835', '505', '49', '27']
did : ['1119'

<ipython-input-19-c23c1cc20f2e>:55: RuntimeWarning: invalid value encountered in true_divide
  bm25_scores_normalize = (bm25_scores - bm25_mean) / bm25_std


did : ['1049'], score : [0.96969697]
did : ['806'], score : [1.]
did = 1067
label pos dids
['1074']
did : ['1074'], score : [1.]
did = 1074
label pos dids
['1067']
did : ['1067'], score : [1.]
did = 1075
label pos dids
['434', '424', '305']
did : ['434'], score : [0.82352941]
did : ['424'], score : [0.82352941]
did : ['305'], score : [0.93333333]
did = 1076
label pos dids
['261', '344']
did : ['261'], score : [0.94117647]
did : ['344'], score : [0.59259259]
did = 1086
label pos dids
['437', '434']
did : ['437'], score : [0.66666667]
did : ['434'], score : [0.94117647]
did = 1111
label pos dids
['1116']
did : ['1116'], score : [0.84210526]
did = 1114
label pos dids
['850']
did : ['850'], score : [0.42105263]
did = 1116
label pos dids
['1111']
did : ['1111'], score : [0.84210526]
did = 1118
label pos dids
['1125']
did : ['1125'], score : [0.96551724]
did = 1120
label pos dids
['1119']
did : ['1119'], score : [0.68965517]
did = 1126
label pos dids
['774', '326', '656', '676']
did : ['774'

did : ['1333'], score : [1.]
did = 1337
label pos dids
['1324', '1253']
did : ['1324'], score : [1.]
did : ['1253'], score : [1.]
did = 1339
label pos dids
['1276', '1298', '1302', '1290']
did : ['1276'], score : [0.875]
did : ['1298'], score : [0.93333333]
did : ['1302'], score : [0.93333333]
did : ['1290'], score : [0.875]
did = 1340
label pos dids
['1278', '1190', '1172', '1232', '1302', '1207', '1230', '1199', '1180', '1187', '1198', '1339', '1250', '1290', '1274', '1347', '1350', '1298', '1236', '1194', '1249', '1276', '1243']
did : ['1278'], score : [0.66666667]
did : ['1190'], score : [0.75]
did : ['1172'], score : [0.36363636]
did : ['1232'], score : [0.85714286]
did : ['1302'], score : [0.6]
did : ['1207'], score : [0.66666667]
did : ['1230'], score : [0.75]
did : ['1199'], score : [0.66666667]
did : ['1180'], score : [0.36363636]
did : ['1187'], score : [0.5]
did : ['1198'], score : [0.75]
did : ['1339'], score : [0.54545455]
did : ['1250'], score : [0.66666667]
did : ['1290'

In [20]:
# print(len(y))
# print(len(y_bm25))
# print(len(y_pred))

multi = MultiLabelBinarizer()
y_multi = multi.fit_transform(y)
y_bm25_multi = multi.transform(y_bm25)
y_pred_multi = multi.transform(y_pred)
y_second_pred_multi = multi.transform(y_second_pred)
y_set_multi = multi.transform(y_sets)
y_binary_multi = multi.transform(y_binary)
y_mixed_multi = multi.transform(y_mixed)


# print(y_multi.shape)
# print(y_bm25_multi.shape)
# print(y_pred_multi.shape)

average = 'macro'
bm25_f1  = f1_score(y_multi, y_bm25_multi ,average=average)
bm25_recall  = recall_score(y_multi, y_bm25_multi ,average=average)
bm25_precision  = precision_score(y_multi, y_bm25_multi ,average=average)

score_f1  = f1_score(y_multi, y_mixed_multi ,average=average)
score_recall  = recall_score(y_multi, y_mixed_multi ,average=average)
score_precision  = precision_score(y_multi, y_mixed_multi ,average=average)

print('score')
print(f'f1:{score_f1}, recall:{score_recall}, precision:{score_precision} ')

print('bm25')
print(f'f1:{bm25_f1}, recall:{bm25_recall}, precision:{bm25_precision} ')

pred_f1  = f1_score(y_multi, y_pred_multi ,average=average)
pred_recall  = recall_score(y_multi, y_pred_multi ,average=average)
pred_precision  = precision_score(y_multi, y_pred_multi ,average=average)

print('preds')
print(f'f1:{pred_f1}, recall:{pred_recall}, precision:{pred_precision} ')

second_pred_f1  = f1_score(y_multi, y_second_pred_multi ,average=average)
second_pred_recall  = recall_score(y_multi, y_second_pred_multi ,average=average)
second_pred_precision  = precision_score(y_multi, y_second_pred_multi ,average=average)

print('second pred')
print(f'f1:{second_pred_f1}, recall:{second_pred_recall}, precision:{second_pred_precision} ')


set_f1  = f1_score(y_multi, y_set_multi ,average=average)
set_recall  = recall_score(y_multi, y_set_multi ,average=average)
set_precision  = precision_score(y_multi, y_set_multi ,average=average)

print('set')
print(f'f1:{set_f1}, recall:{set_recall}, precision:{set_precision} ')


binary_f1  = f1_score(y_multi, y_binary_multi ,average=average)
binary_recall  = recall_score(y_multi, y_binary_multi ,average=average)
binary_precision  = precision_score(y_multi, y_binary_multi ,average=average)

print('binary')
print(f'f1:{binary_f1}, recall:{binary_recall}, precision:{binary_precision} ')


mixed_f1  = f1_score(y_multi, y_mixed_multi ,average=average)
mixed_recall  = recall_score(y_multi, y_mixed_multi ,average=average)
mixed_precision  = precision_score(y_multi, y_mixed_multi ,average=average)

print('mixed')
print(f'f1:{mixed_f1}, recall:{mixed_recall}, precision:{mixed_precision} ')

score
f1:0.0, recall:0.0, precision:0.0 
bm25
f1:0.4987896657452928, recall:0.7956043868764883, precision:0.4316828775792857 
preds
f1:0.21346016156275824, recall:0.1979947904017098, precision:0.2940751445086705 
second pred
f1:0.38414619520498916, recall:0.37012427506944673, precision:0.4920382603908615 
set
f1:0.26278960564917725, recall:0.24299713047162894, precision:0.37668593448940274 
binary
f1:0.0, recall:0.0, precision:0.0 
mixed
f1:0.0, recall:0.0, precision:0.0 


/home/nlplab/harry/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:860: UserWarning: unknown class(es) ['1', '1011', '105', '1056', '1068', '1082', '1086', '1087', '1093', '110', '1118', '113', '1130', '1142', '1146', '1147', '1151', '1153', '1155', '1157', '1159', '1161', '1165', '117', '119', '1191', '1206', '121', '1239', '1261', '1271', '1301', '1334', '1341', '1352', '1354', '1366', '1375', '1390', '1401', '142', '147', '150', '152', '157', '185', '189', '199', '201', '21', '215', '219', '225', '242', '26', '267', '268', '273', '283', '295', '299', '3', '301', '310', '319', '333', '337', '342', '343', '351', '365', '371', '381', '387', '39', '392', '403', '411', '421', '441', '449', '463', '471', '472', '474', '484', '501', '508', '51', '510', '516', '522', '530', '532', '54', '544', '553', '568', '576', '585', '591', '597', '598', '6', '602', '603', '611', '628', '631', '633', '653', '675', '722', '725', '726', '73', '730', '733', '741', '746', '755', '764',

In [23]:
print('pos score')
get_static_info(all_pos_scores)
print(len(all_pos_scores))
n, bins, patches = plt.hist(all_pos_scores)
plt.show()

print('all score')
get_static_info(all_scores)
print(len(all_scores))

n, bins, patches = plt.hist(all_scores)
plt.show()

pos score


NameError: name 'get_static_info' is not defined

In [22]:
print(len(train_binary_pairs))
print(train_pos_count, train_neg_count)
print(train_pos_count / len(train_binary_pairs))
train_binary_pairs[:10]

0
0 0


ZeroDivisionError: division by zero

In [ ]:
import pickle
with open('./data/train_binary_pairs', 'wb') as handle:
    pickle.dump(train_binary_pairs, handle, protocol=pickle.HIGHEST_PROTOCOL)